# Github: initial fetch

In [2]:
import os
import json
import shutil
from pathlib import Path

import requests
import requests_cache

from nya.config import settings

In [3]:
settings.dict()

{'root_dir': PosixPath('/home/michael/Documents/no-ur-awesome/py'),
 'github_token': SecretStr('**********'),
 'github_cache_path': PosixPath('/home/michael/Documents/no-ur-awesome/py/data/raw/github_cache.db'),
 'github_cache_expires': 604800.0}